In [1]:
import time
import requests
import os
def get_vin_decoder_data(row):
    try:
       
        url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/'
        response = requests.post(url,row)
        return response
    #Sometimes it will return an error. This will catch the error and cause it to wait for 60 seconds before trying again
    except Exception as e:
        print ("connection Error, Retrying in 60 seconds")
        time.sleep(60)
        url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/'
        response = requests.post(url,row)
        return response
    else:
        raise e
    

In [ ]:
# Prep the file for processing with NHTSA API. 
# You wll need the vin and the year for the best results
import pandas as pd
#You can import your file or you can bring in as an object
results_final = pd.read_csv('Filename.csv')
data_to_process=pd.DataFrame({"vin":results_final.VIN,"year":results_final.YEAR})
data_to_process = pd.DataFrame(results_final)
data_to_process["YEAR"]=data_to_process["YEAR"].astype(str)
data_to_process


In [ ]:
#Not necessary but if you want to keep a subset of records
data_to_process= data_to_process.reset_index(drop=True)
data_to_process = data_to_process.loc[0:99999]
data_to_process

In [13]:
#loop the records through the API in chunks since the API supports it. 
import json 
from pandas import json_normalize
from more_itertools import sliced
CHUNK_SIZE = 10

index_slices = sliced(range(len(data_to_process)), CHUNK_SIZE)
chunk=pd.DataFrame()

In [15]:

i=0
line_to_process=' ';
vin_warehouse=pd.DataFrame()
chunk=pd.DataFrame()
for index_slice in index_slices:
    try:
        i=i+1
        #format the line to be processed as dictated by API requirements. You can put up to 10 values in a single request
        line_to_process+=data_to_process.iloc[index_slice]['VIN']+","+data_to_process.loc[index_slice]['YEAR']+";"
        line_to_process=line_to_process.to_string(index=False)
        #dictate the style of output from the API
        post_fields = {'format': 'json', 'data':line_to_process};
        #call the data from the API
        data=get_vin_decoder_data(post_fields);
        #convert data from json to text 
        get_data =  json.loads(data.text)
        load_data = pd.DataFrame(json_normalize(get_data['Results']))
        #Append the records into a dataframe
        if i == 1:
            vin_warehouse=load_data
        else:
            vin_warehouse = pd.concat([vin_warehouse, load_data], ignore_index=True)
        #Null out the line to process to avoid resubmitting vins that were already processed
        line_to_process=' '
    except KeyError:
        continue

In [16]:
#Optional: You can append your data to Snowflake. 
cursor, conn  = Snowflake().connect_to_snowflake()
snowflake_curr = cursor
vin_warehouse.columns = map(str.upper, vin_warehouse.columns)

In [17]:
vin_warehouse_final = vin_warehouse[['VIN',
 'BATTERYINFO',
 'BATTERYTYPE',
 'BEDTYPE',
 'BODYCABTYPE',
 'BODYCLASS',
 'ENGINECYLINDERS',
 'DESTINATIONMARKET',
 'DISPLACEMENTCC',
 'DISPLACEMENTCI',
 'DISPLACEMENTL',
 'DOORS',
 'DRIVETYPE',
 'DRIVERASSIST',
 'ENGINECYCLES',
 'ENGINEMODEL',
 'ENGINEKW',
 'ENTERTAINMENTSYSTEM',
 'FUELTYPEPRIMARY',
 'GVWR',
 'MAKE',
 'MANUFACTURER',
 'MODEL',
 'MODELYEAR',
 'PLANTCITY',
 'SEATS',
 'SERIES',
 'STEERINGLOCATION',
 'TRANSMISSIONSTYLE',
 'TRIM',
 'VEHICLETYPE',
 'WINDOWS',
 'AXLES',
 'BRAKESYSTEMTYPE',
 'BATTERYCELLS',
 'BEDLENGTHIN',
 'BRAKESYSTEMDESC',
 'CURBWEIGHTLB',
 'AIRBAGLOCCURTAIN',
 'AIRBAGLOCSEATCUSHION',
 'BATTERYA',
 'BATTERYV',
 'BATTERYKWH',
 'WHEELBASETYPE',
 'SEATROWS',
 'VALVETRAINDESIGN',
 'TRANSMISSIONSPEEDS',
 'ENGINECONFIGURATION',
 'AIRBAGLOCFRONT',
 'FUELTYPESECONDARY',
 'FUELINJECTIONTYPE',
 'AIRBAGLOCKNEE',
 'ENGINEHP',
 'EVDRIVEUNIT',
 'PLANTCOUNTRY',
 'PLANTCOMPANYNAME',
 'PLANTSTATE',
 'PRETENSIONER',
 'SEATBELTSALL',
 'ADAPTIVECRUISECONTROL',
 'ADAPTIVEHEADLIGHTS',
 'ABS',
 'CIB',
 'BLINDSPOTMON',
 'NCSABODYTYPE',
 'NCSAMAKE',
 'NCSAMODEL',
 'ESC',
 'TRACTIONCONTROL',
 'FORWARDCOLLISIONWARNING',
 'LANEDEPARTUREWARNING',
 'LANEKEEPSYSTEM',
 'REARVISIBILITYSYSTEM',
 'PARKASSIST',
 'AIRBAGLOCSIDE',
 'TRIM2',
 'SERIES2',
 'WHEELBASESHORT',
 'WHEELBASELONG',
 'NOTE',
 'WHEELS',
 'TRAILERTYPE',
 'TRAILERBODYTYPE',
 'TRAILERLENGTH',
 'WHEELSIZEFRONT',
 'WHEELSIZEREAR',
 'OTHERRESTRAINTSYSTEMINFO',
 'COOLINGTYPE',
 'ENGINEHP_TO',
 'ELECTRIFICATIONLEVEL',
 'CHARGERLEVEL',
 'CHARGERPOWERKW',
 'OTHERENGINEINFO',
 'BATTERYA_TO',
 'BATTERYV_TO',
 'BATTERYKWH_TO',
 'TURBO',
 'BASEPRICE',
 'BATTERYMODULES',
 'BATTERYPACKS',
 'TOPSPEEDMPH',
 'SUGGESTEDVIN',
 'ERRORCODE',
 'POSSIBLEVALUES',
 'AXLECONFIGURATION',
 'ENGINEMANUFACTURER',
 'BUSLENGTH',
 'BUSFLOORCONFIGTYPE',
 'BUSTYPE',
 'OTHERBUSINFO',
 'CUSTOMMOTORCYCLETYPE',
 'MOTORCYCLESUSPENSIONTYPE',
 'MOTORCYCLECHASSISTYPE',
 'OTHERMOTORCYCLEINFO',
 'OTHERTRAILERINFO',
 'ADDITIONALERRORTEXT',
 'MANUFACTURERID',
 'CASHFORCLUNKERS',
 'TRACKWIDTH',
 'TPMS',
 'ACTIVESAFETYSYSNOTE',
 'DYNAMICBRAKESUPPORT',
 'PEDESTRIANAUTOMATICEMERGENCYBRAKING',
 'AUTOREVERSESYSTEM',
 'AUTOMATICPEDESTRIANALERTINGSOUND',
 'CAN_AACN',
 'EDR',
 'KEYLESSIGNITION',
 'DAYTIMERUNNINGLIGHT',
 'LOWERBEAMHEADLAMPLIGHTSOURCE',
 'SEMIAUTOMATICHEADLAMPBEAMSWITCHING',
 'ADAPTIVEDRIVINGBEAM',
 'SAEAUTOMATIONLEVEL',
 'SAEAUTOMATIONLEVEL_TO',
 'REARCROSSTRAFFICALERT',
 'GCWR',
 'GCWR_TO',
 'NCSANOTE',
 'NCSAMAPPINGEXCEPTION',
 'NCSAMAPEXCAPPROVEDON',
 'NCSAMAPEXCAPPROVEDBY',
 'GVWR_TO',
 'ERRORTEXT',
 'MAKEID',
 'MODELID',
 'REARAUTOMATICEMERGENCYBRAKING',
 'BLINDSPOTINTERVENTION',
 'LANECENTERINGASSISTANCE',
 'NONLANDUSE',
 'VEHICLEDESCRIPTOR']]

In [20]:
import snowflake.connector
import sys
from snowflake.connector.pandas_tools import write_pandas
cursor, conn  = Snowflake().connect_to_snowflake()
snowflake_curr = cursor

In [21]:
#Write Results to Table in Snowflake
if len(vin_warehouse_final) >0:
    success, nchunks, nrows, _ = write_pandas(conn, vin_warehouse_final, table_name ='TABLE', 
    database='DATABASE', schema='SCHEMA')